In [255]:
import pandas as pd

class Escola:
    def __init__(self, URE, USE, MUNICIPIO, COD_ESCOLA, COD_MEC, COD_SETOR, ESCOLA, COD_DISCIPLINA, CH_HORARIA, VAGAS, lista_professores):
        self.URE = URE
        self.USE = USE
        self.MUNICIPIO = MUNICIPIO
        self.COD_ESCOLA = COD_ESCOLA
        self.COD_MEC = COD_MEC
        self.COD_SETOR = COD_SETOR
        self.ESCOLA = ESCOLA
        self.COD_DISCIPLINA = COD_DISCIPLINA
        self.CH_HORARIA = CH_HORARIA
        self.VAGAS = VAGAS
        self.lista_professores = lista_professores

# Carregar o DataFrame
base_escolas = pd.read_excel('escolas.xlsx')

# Dicionário para armazenar os objetos Escola
escolas_dict = {}

# Percorrer as linhas do DataFrame
for indice, linha in base_escolas.iterrows():
    # Gerar o nome do objeto
    nome_objeto = f"escola_{linha['COD_ESCOLA']}_{linha['COD_DISCIPLINA']}"
    
    # Criar um objeto Escola para cada linha
    escola = Escola(
        linha['URE'],
        linha['USE'],
        linha['MUNICIPIO'],
        linha['COD_ESCOLA'],
        linha['COD_MEC'],
        linha['COD_SETOR'],
        linha['ESCOLA'],
        linha['COD_DISCIPLINA'],
        linha['CH_HORARIA'],
        linha['VAGAS'],
        lista_professores = []
        # LISTA DE PROFESSORES (VAZIA)
    )
    

    # Adicionar o objeto ao dicionário usando o nome como chave
    escolas_dict[nome_objeto] = escola

for nome_objeto, escola in escolas_dict.items():
    print(f"Nome do objeto: {nome_objeto}, VAGAS: {escola.VAGAS}, Disciplina {escola.COD_DISCIPLINA}")       


# CONTRATAÇÃO POR PREFERENCIA
# SUBTRAI NUMERO DE VAGAS

# CONFERIR REPETIÇÃO NO PRIMEIRO ELEMENTO DAS OUTRAS LISTAS
# REMOÇÃO DO PROFESSOR DE OUTRAS LISTAS



Nome do objeto: escola_1_1, VAGAS: 2, Disciplina 1
Nome do objeto: escola_2_2003, VAGAS: 3, Disciplina 2003
Nome do objeto: escola_3_2002, VAGAS: 3, Disciplina 2002
Nome do objeto: escola_4_2003, VAGAS: 1, Disciplina 2003
Nome do objeto: escola_10_2002, VAGAS: 3, Disciplina 2002
Nome do objeto: escola_10_1, VAGAS: 1, Disciplina 1


In [256]:
base_professores = pd.read_excel('prof.xlsx')

# base_professores = base_professores.sort_values(by=['DATA_EXERCICIO', 'SEGUNDA_HABILITACAO', 'area_conhecimento', 'INTERNO', 'EFETIVO'], ascending=[True, False, False, False, False])

base_professores = base_professores.sort_values(by='DATA_EXERCICIO', ascending=True)
base_professores = base_professores.sort_values(by='SEGUNDA_HABILITACAO', ascending=False)
base_professores = base_professores.sort_values(by='area_conhecimento', ascending=False)
#base_professores = base_professores.sort_values(by='INTERNO', ascending=False)
base_professores = base_professores.sort_values(by='EFETIVO', ascending=False)

professores_internos = base_professores[base_professores['INTERNO'] == 'SIM']
display(professores_internos)

global professores_lotados

professores_lotados = pd.DataFrame(columns=['URE', 'USE', 'MUNICIPIO', 'COD_ESCOLA', 'COD_MEC','COD_SETOR', 'ESCOLA','COD_DISCIPLINA'])


,URE,USE,MUNICIPIO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,MATRICULA,...,ANO_LETIVO,DISCIPLINA,CH_SEMANAL,valor,area_conhecimento,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5
8,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,1,Professor 9,5951384,...,2023,MATEMATICA,3,1,SIM,1,8,3.0,9.0,2.0
9,01A URE - BRAGANCA,NaN,BRAGANCA,10,15057046,1920914340000,EEEFM DO ROCHA,2002,Professor 10,5951384,...,2023,HISTORIA,3,9,SIM,10,9,8.0,3.0,5.0
0,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,2003,Professor 1,5948590,...,2023,GEOGRAFIA,23,7,SIM,1,2,3.0,4.0,5.0
2,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,Professor 3,5973168,...,2023,HISTORIA,4,6,SIM,3,3,5.0,2.0,7.0
7,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,1,Professor 8,5923338,...,2023,MATEMATICA,3,7,NÃO,1,9,8.0,7.0,6.0
1,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920930000000,EE BENEDITO CARDOSO DE ATAYDE,2003,Professor 2,5936943,...,2023,GEOGRAFIA,5,4,NÃO,2,3,4.0,5.0,6.0
3,01A URE - BRAGANCA,NaN,BRAGANCA,4,15057291,1920913530000,EEEF PATALINO,2002,Professor 4,5971966,...,2023,HISTORIA,3,1,NÃO,4,3,8.0,5.0,10.0


In [257]:
# RODANDO APENAS PROFESSORES INTERNOS
for nome_objeto, escola in escolas_dict.items():

    for index, professor in professores_internos.iterrows():
        cod_disciplina = professor['COD_DISCIPLINA']
        agrupamento = professor['MUNICIPIO']
        cod_escola = professor['COD_ESCOLA']
        carga_horaria = professor['JORNADA']
        
        if (escola.COD_DISCIPLINA == cod_disciplina) and (cod_escola == escola.COD_ESCOLA) and (escola.MUNICIPIO == agrupamento):
            servidor = professor['SERVIDOR']
            # Verifica se o professor já foi adicionado à lista da escola
            if servidor not in escola.lista_professores:
                escola.lista_professores.append(servidor)

        else:
            pass

for nome_objeto, escola in escolas_dict.items():
    print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")
       
# FAZENDO A LOTACAO
for nome_objeto, escola in escolas_dict.items():
    #print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")
    while escola.VAGAS > 0 and escola.lista_professores:
        # Obtendo o primeiro professor da lista
        professor = escola.lista_professores[0]
        
        # Criando um dicionário com os dados do professor e da escola
        dados = {
            'URE': escola.URE,
            'USE': escola.USE,
            'MUNICIPIO': escola.MUNICIPIO,
            'COD_ESCOLA': escola.COD_ESCOLA,
            'COD_MEC': escola.COD_MEC,
            'COD_SETOR': escola.COD_SETOR,
            'ESCOLA': escola.ESCOLA,
            'COD_DISCIPLINA': escola.COD_DISCIPLINA,
            'CH_HORARIA': escola.CH_HORARIA,
            'Professor': professor  # Adiciona um professor por vez
        }
        # Adicionando os dados do professor ao DataFrame professores_lotados
        professores_lotados = professores_lotados._append(dados, ignore_index=True)
        
        for index, servidor in professores_internos.iterrows():
            if professor == servidor['SERVIDOR']:
                cod = index
                base_professores.drop(cod, inplace=True)
        # Removendo o professor da lista
        escola.lista_professores.pop(0)
        
        # Subtraindo 1 da quantidade de vagas
        escola.VAGAS -= 1
        
        # Exibindo informações

display(professores_lotados)

for nome_objeto, escola in escolas_dict.items():
    print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")

Nome do objeto: escola_1_1, PROFESSORES: ['Professor 9', 'Professor 8'], VAGAS: 2
Nome do objeto: escola_2_2003, PROFESSORES: ['Professor 2'], VAGAS: 3
Nome do objeto: escola_3_2002, PROFESSORES: ['Professor 3'], VAGAS: 3
Nome do objeto: escola_4_2003, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_10_2002, PROFESSORES: ['Professor 10'], VAGAS: 3
Nome do objeto: escola_10_1, PROFESSORES: [], VAGAS: 1


,URE,USE,MUNICIPIO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,CH_HORARIA,Professor
0,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920914550000,EEEM ANDRE ALVES,1,GRANDE,Professor 9
1,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920914550000,EEEM ANDRE ALVES,1,GRANDE,Professor 8
2,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920928040000,EEEM BENEDITO CARDOSO DE ATHAYDE,2003,GRANDE,Professor 2
3,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,GRANDE,Professor 3
4,01A URE - BRAGANCA,NaN,BRAGANCA,10,15057046,1920914340000,EEEFM DO ROCHA,2002,GRANDE,Professor 10


Nome do objeto: escola_1_1, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2003, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_3_2002, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_4_2003, PROFESSORES: [], VAGAS: 1
Nome do objeto: escola_10_2002, PROFESSORES: [], VAGAS: 2
Nome do objeto: escola_10_1, PROFESSORES: [], VAGAS: 1


In [258]:
display(base_professores)

# CRIANDO AS LISTAS PARA LOTAÇÃO
for nome_objeto, escola in escolas_dict.items():

    for index, professor in base_professores.iterrows():
        cod_disciplina = professor['COD_DISCIPLINA']
        agrupamento = professor['MUNICIPIO']
        cod_escola = professor['COD_ESCOLA']
        ch_prof = professor['JORNADA']

        if escola.CH_HORARIA == 'GRANDE':
            ch_escola = 40
        else:
            ch_escola = 20

        if (escola.COD_DISCIPLINA == cod_disciplina)and (escola.MUNICIPIO == agrupamento):
            if ch_prof == 40 and ch_escola == 20:
                pass
            else:
                servidor = professor['SERVIDOR']
                # Verifica se o professor já foi adicionado à lista da escola
                if servidor not in escola.lista_professores:
                    escola.lista_professores.append(servidor)

        else:
            pass
        
        #print(nome_objeto, escola.lista_professores)
        # ESCOLA.LISTA ADICIONA CADA PROFESSOR QUE CUMPRA OS REQUISITOS
        # LISTA ATUALIZADA COM RANKING (3, 9, 5, 6, 7, 8)



for nome_objeto, escola in escolas_dict.items():
    print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")

,URE,USE,MUNICIPIO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,MATRICULA,...,ANO_LETIVO,DISCIPLINA,CH_SEMANAL,valor,area_conhecimento,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5
10,01A URE - BRAGANCA,NaN,BRAGANCA,4,15057291,1920913530000,EEEF PATALINO,2002,Professor 11,5951385,...,2023,HISTORIA,3,9,SIM,3,9,8.0,7.0,6.0
0,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,2003,Professor 1,5948590,...,2023,GEOGRAFIA,23,7,SIM,1,2,3.0,4.0,5.0
5,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920930000000,EE BENEDITO CARDOSO DE ATAYDE,2002,Professor 6,5936943,...,2023,HISTORIA,3,5,NÃO,10,6,7.0,8.0,5.0
11,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920930000000,EE BENEDITO CARDOSO DE ATAYDE,1,Professor 12,5951386,...,2023,MATEMATICA,3,9,NAO,1,10,NaN,NaN,NaN
6,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,Professor 7,5974417,...,2023,HISTORIA,3,5,SIM,2,4,7.0,6.0,10.0
4,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,2003,Professor 5,5922696,...,2023,GEOGRAFIA,3,9,SIM,4,2,8.0,10.0,5.0
12,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920910000000,EE ANDRE ALVES,2003,Professor 13,5922696,...,2023,GEOGRAFIA,3,9,SIM,4,2,8.0,10.0,5.0
3,01A URE - BRAGANCA,NaN,BRAGANCA,4,15057291,1920913530000,EEEF PATALINO,2002,Professor 4,5971966,...,2023,HISTORIA,3,1,NÃO,4,3,8.0,5.0,10.0


Nome do objeto: escola_1_1, PROFESSORES: ['Professor 12'], VAGAS: 0
Nome do objeto: escola_2_2003, PROFESSORES: ['Professor 1', 'Professor 5', 'Professor 13'], VAGAS: 2
Nome do objeto: escola_3_2002, PROFESSORES: ['Professor 11', 'Professor 6', 'Professor 7', 'Professor 4'], VAGAS: 2
Nome do objeto: escola_4_2003, PROFESSORES: ['Professor 5', 'Professor 13'], VAGAS: 1
Nome do objeto: escola_10_2002, PROFESSORES: ['Professor 11', 'Professor 6', 'Professor 7', 'Professor 4'], VAGAS: 2
Nome do objeto: escola_10_1, PROFESSORES: ['Professor 12'], VAGAS: 1


In [259]:
def desempate():
    for nome_objeto, escola in escolas_dict.items():
        if escola.lista_professores:
            professor = escola.lista_professores[0]
            carencia = escola.VAGAS - len(escola.lista_professores) 
            
            for objeto_verificado, escola_verificada in escolas_dict.items():
                if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola

                    if professor in escola_verificada.lista_professores:
                        if escola.VAGAS == 0:
                            escola.lista_professores.pop(0)
                        else:
                            indice = escola_verificada.lista_professores.index(professor)
                            carencia_verificada = escola_verificada.VAGAS - len(escola_verificada.lista_professores)

                            if indice > 0: # TOP 3 PREFERENCIAS
                                print(carencia_verificada)
                                if carencia_verificada > -1:
                                    escola.lista_professores.pop(0)
                                    print(f'Popando da {nome_objeto}, a {objeto_verificado} tem carencia {carencia_verificada}')
                                else:
                                    escola_verificada.lista_professores.pop(indice)
                                                        
                            else:
                                if escola_verificada.VAGAS == 0:
                                    escola_verificada.lista_professores.pop(0)
                                else:
                                    preferencias = []
                                    for index, servidor in base_professores.iterrows():
                                        if professor == servidor['SERVIDOR']:
                                            preferencias = [base_professores.at[index, f'PREFERENCIA {i}'] for i in range(1, 6)]
                                            break
                                
                                    try:
                                        escola1 = preferencias.index(escola.COD_ESCOLA)
                                    except ValueError:
                                        print(f"Valor {escola.COD_ESCOLA} não encontrado em preferencias.")
                                        try:
                                            escola2 = preferencias.index(escola_verificada.COD_ESCOLA)
                                        except ValueError:
                                            print(f"Valor {escola_verificada.COD_ESCOLA} não encontrado em preferencias.")
                                            if carencia_verificada == 0 and carencia > 0:
                                                escola_verificada.lista_professores.pop(0)
                                                continue  # Passa para a próxima iteração do loop interno
                                            elif carencia_verificada == 0 and carencia < 0:
                                                escola.lista_professores.pop(0)
                                                continue
                                            elif carencia_verificada > 0 and carencia == 0:
                                                escola.lista_professores.pop(0)
                                                continue
                                            elif carencia_verificada < 0 and carencia == 0:
                                                escola_verificada.lista_professores.pop(0)
                                                continue
                                            elif carencia_verificada == 0 and carencia == 0:
                                                escola_verificada.lista_professores.pop(0)
                                                continue
                                            escola_verificada.lista_professores.pop(0)

                                        escola.lista_professores.pop(0)


    print('-'*20)
    for nome_objeto, escola in escolas_dict.items():
        print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")
    print('-'*20)


In [260]:
def lotacaoGeral():
    global professores_lotados
    
    while True:
        # Mantém a flag para verificar se houve alocação em algum loop
        alocou = False
        
        desempate()
        for nome_objeto, escola in escolas_dict.items():
            # Verifica se a lista de professores não está vazia
            if escola.VAGAS > 0 and escola.lista_professores:
                # Obtendo o primeiro professor da lista
                professor = escola.lista_professores[0]
                print(professor)
                # Criando um dicionário com os dados da escola e do professor
                dados = {
                    'URE': escola.URE,
                    'USE': escola.USE,
                    'MUNICIPIO': escola.MUNICIPIO,
                    'COD_ESCOLA': escola.COD_ESCOLA,
                    'COD_MEC': escola.COD_MEC,
                    'COD_SETOR': escola.COD_SETOR,
                    'ESCOLA': escola.ESCOLA,
                    'COD_DISCIPLINA': escola.COD_DISCIPLINA,
                    'CH_HORARIA': escola.CH_HORARIA,
                    'Professor': professor  # Adiciona um professor por vez
                }
                
                # Adicionando os dados do professor ao DataFrame professores_lotados
                professores_lotados = professores_lotados._append(dados, ignore_index=True)
                
                # Removendo o professor alocado da lista de professores
                escola.lista_professores.pop(0)
                
                # Verificações e ações adicionais dentro do while
                
                for objeto_verificado, escola_verificada in escolas_dict.items():
                    if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola              
                        if professor in escola_verificada.lista_professores:
                            indice = escola_verificada.lista_professores.index(professor)
                            escola_verificada.lista_professores.pop(indice)
                
                # Subtraindo 1 da quantidade de vagas
                escola.VAGAS -= 1
                
                # Excluindo o professor alocado do DataFrame base_professores
                for index, servidor in base_professores.iterrows():
                    if professor == servidor['SERVIDOR']:
                        base_professores.drop(index, inplace=True)
                        break  # Interrompe o loop depois de encontrar e remover o professor
                
                # Marca que houve alocação nesta iteração do loop
                alocou = True
        
        # Se nenhum professor foi alocado em nenhum loop, encerra o processo
        if not alocou:
            break


In [261]:

for nome_objeto, escola in escolas_dict.items():
    for nome_objeto, escola in escolas_dict.items():
        print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")
    # Iterando sobre cada professor na lista de professores da escola
    while escola.VAGAS >= 0 and escola.lista_professores:
        #desempate()
        lotacaoGeral()

Nome do objeto: escola_1_1, PROFESSORES: ['Professor 12'], VAGAS: 0
Nome do objeto: escola_2_2003, PROFESSORES: ['Professor 1', 'Professor 5', 'Professor 13'], VAGAS: 2
Nome do objeto: escola_3_2002, PROFESSORES: ['Professor 11', 'Professor 6', 'Professor 7', 'Professor 4'], VAGAS: 2
Nome do objeto: escola_4_2003, PROFESSORES: ['Professor 5', 'Professor 13'], VAGAS: 1
Nome do objeto: escola_10_2002, PROFESSORES: ['Professor 11', 'Professor 6', 'Professor 7', 'Professor 4'], VAGAS: 2
Nome do objeto: escola_10_1, PROFESSORES: ['Professor 12'], VAGAS: 1
-1
Valor 10 não encontrado em preferencias.
--------------------
Nome do objeto: escola_1_1, PROFESSORES: [], VAGAS: 0
Nome do objeto: escola_2_2003, PROFESSORES: ['Professor 1', 'Professor 13'], VAGAS: 2
Nome do objeto: escola_3_2002, PROFESSORES: ['Professor 11', 'Professor 6', 'Professor 7', 'Professor 4'], VAGAS: 2
Nome do objeto: escola_4_2003, PROFESSORES: ['Professor 5', 'Professor 13'], VAGAS: 1
Nome do objeto: escola_10_2002, PROF

In [262]:

professores_lotados = professores_lotados.sort_values(by='COD_ESCOLA', ascending=True)
display(professores_lotados)
#professores_lotados.to_excel('LotacaoProfessores.xlsx', index=False)

# with pd.ExcelWriter('lotacao.xlsx') as writer:
#     professores_lotados.to_excel(writer, sheet_name='Lotação', index=False)
#     base_professores.to_excel(writer, sheet_name='Não Lotados', index=False)

,URE,USE,MUNICIPIO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,CH_HORARIA,Professor
0,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920914550000,EEEM ANDRE ALVES,1,GRANDE,Professor 9
1,01A URE - BRAGANCA,NaN,BRAGANCA,1,15055582,1920914550000,EEEM ANDRE ALVES,1,GRANDE,Professor 8
2,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920928040000,EEEM BENEDITO CARDOSO DE ATHAYDE,2003,GRANDE,Professor 2
5,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920928040000,EEEM BENEDITO CARDOSO DE ATHAYDE,2003,GRANDE,Professor 1
10,01A URE - BRAGANCA,NaN,BRAGANCA,2,15550079,1920928040000,EEEM BENEDITO CARDOSO DE ATHAYDE,2003,GRANDE,Professor 13
3,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,GRANDE,Professor 3
6,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,GRANDE,Professor 11
11,01A URE - BRAGANCA,NaN,BRAGANCA,3,15056651,1920932310000,EEEF JOAO PAULO II,2002,GRANDE,Professor 7
7,01A URE - BRAGANCA,NaN,BRAGANCA,4,15057291,1920913530000,EEEF PATALINO,2003,PEQUENA,Professor 5
4,01A URE - BRAGANCA,NaN,BRAGANCA,10,15057046,1920914340000,EEEFM DO ROCHA,2002,GRANDE,Professor 10


In [263]:
display(base_professores)

,URE,USE,MUNICIPIO,COD_ESCOLA,COD_MEC,COD_SETOR,ESCOLA,COD_DISCIPLINA,SERVIDOR,MATRICULA,...,ANO_LETIVO,DISCIPLINA,CH_SEMANAL,valor,area_conhecimento,PREFERENCIA 1,PREFERENCIA 2,PREFERENCIA 3,PREFERENCIA 4,PREFERENCIA 5


In [264]:
# #

# for nome_objeto, escola in escolas_dict.items():
#     try:
#         professor = escola.lista_professores[0]
#         carencia = len(escola.lista_professores) - escola.VAGAS
#         print("Lista de professores:", escola.lista_professores)
#         print("Primeiro professor:", professor)
        
#         for objeto_verificado, escola_verificada in escolas_dict.items():
#             if objeto_verificado != nome_objeto:  # Garante que não estamos verificando a mesma escola

#                 if professor in escola_verificada.lista_professores:
#                     indice = escola_verificada.lista_professores.index(professor)
#                     carencia_verificada = len(escola_verificada.lista_professores) - escola_verificada.VAGAS
#                     print(f"O professor {professor} está presente na lista de professores de {objeto_verificado} no índice {indice} e com vagas: {escola_verificada.VAGAS} e {len(escola_verificada.lista_professores)} disponiveis")

#                     if indice > 0:
#                         if carencia_verificada != 0:
#                             escola_verificada.lista_professores.pop(indice)
#                         elif carencia_verificada == 0 and carencia < 0:
#                             escola.lista_professores.pop(0)
                    
#                     else:

#                         for index, servidor in base_professores.iterrows():
#                             if professor == servidor['SERVIDOR']:
#                                 preferencias = [base_professores.at[index, f'PREFERENCIA {i}'] for i in range(1, 6)]

#                                 try:
#                                     escola1 = preferencias.index(escola.COD_ESCOLA)
#                                 except ValueError:
#                                     try:
#                                         escola2 = preferencias.index(escola_verificada.COD_ESCOLA)
#                                     except ValueError:
#                                         if carencia_verificada != 0:
#                                             escola_verificada.lista_professores.pop(indice)
#                                         elif carencia_verificada == 0 and carencia < 0:
#                                             escola.lista_professores.pop(0)
#                                     escola.lista_professores.pop(0)              
#                                 try:
#                                     escola2 = preferencias.index(escola_verificada.COD_ESCOLA)
#                                 except ValueError:
#                                     escola_verificada.lista_professores.pop(indice)

#                         print(preferencias, escola.COD_ESCOLA, escola_verificada.COD_ESCOLA)


#     except IndexError:
#         pass  # Trata exceção caso a lista de professores esteja vazia ou não tenha índice 0


# for nome_objeto, escola in escolas_dict.items():
#     print(f"Nome do objeto: {nome_objeto}, PROFESSORES: {escola.lista_professores}, VAGAS: {escola.VAGAS}")

